In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import torch.nn as nn
import torch

from data_loader import GraphCancerMolecules, get_datasets
from data_loader import make_timestamp, split_data
from train import run_a_train_epoch, run_an_eval_epoch
import seaborn as sns
from torch.nn import ELU
from fcd_torch import FCD

from rdkit import Chem
from torch.utils.data import DataLoader

from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
sns.set_context('talk')

Using backend: pytorch


In [2]:
cpdb = GraphCancerMolecules(purpose='cpdb_full', use_ddb_negatives=True, use_ccri_negatives=True, out_feats=2)

TypeError: __init__() got an unexpected keyword argument 'use_ccri_negatives'

In [3]:
df = pd.read_csv('../data/ccri_smiles.csv')
df['smiles'] = GraphCancerMolecules.smiles_standardize(df['smiles'])
df = df[df['smiles'].notnull()]
df.loc[df['negative_carc'] == 1 ,'carcinogenic_label'] = 'negative'
df.loc[df['negative_stringent_carc'] == 1 ,'carcinogenic_label'] = 'negative stringent'
df.loc[df['positive_carc'] == 1 ,'carcinogenic_label'] = 'positive'
df.loc[df['positive_stringent_carc'] == 1 ,'carcinogenic_label'] = 'positive stringent'

df.loc[df['negative_mut'] == 1 ,'mutagenic_label'] = 'negative'
df.loc[df['negative_stringent_mut'] == 1 ,'mutagenic_label'] = 'negative stringent'
df.loc[df['positive_mut'] == 1 ,'mutagenic_label'] = 'positive'
df.loc[df['positive_stringent_mut'] == 1 ,'mutagenic_label'] = 'positive stringent'

assert all(df[df['positive_stringent_mut'] == 1]['positive_mut'])
assert all(df[df['negative_stringent_mut'] == 1]['negative_mut'])
assert all(df[df['positive_stringent_carc'] == 1]['positive_carc'])
assert all(df[df['negative_stringent_carc'] == 1]['negative_carc'])

# fill na for columns for calculations
cols_to_fill = ['ratio_of_positive_carc', 'number_of_experiments_carc', 'ratio_of_positive_mut', 'number_of_experiments_mut']
# df.loc[:, cols_to_fill] = df.loc[:, cols_to_fill].fillna(0)

df.tail()

,ratio_of_positive_carc,number_of_experiments_carc,cas_number,substance_name,use_carc,positive_carc,positive_stringent_carc,negative_carc,negative_stringent_carc,ratio_of_positive_mut,number_of_experiments_mut,use_mut,positive_mut,positive_stringent_mut,negative_mut,negative_stringent_mut,positive,positive_stringent,negative,negative_stringent,use,smiles,carcinogenic_label,mutagenic_label
8818,NaN,NaN,99766-47-9,"2,6-DIETHYLNITROSOBENZENE",NaN,NaN,NaN,NaN,NaN,1.0,4.0,NaN,1.0,1.0,0.0,0.0,1,1,0,0,",",CCc1cccc(CC)c1N=O,NaN,positive stringent
8819,NaN,NaN,99803-60-8,NITROSOFENITROTHION,NaN,NaN,NaN,NaN,NaN,1.0,4.0,NaN,1.0,1.0,0.0,0.0,1,1,0,0,",",COP(=S)(OC)Oc1ccc(N=O)c(C)c1,NaN,positive stringent
8820,NaN,NaN,999-29-1,N-DIAZOACETYLGLYCINE ETHYL ESTER,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,1.0,1.0,0.0,0.0,1,1,0,0,",",CCOC(=O)CNC(=O)C=[N+]=[N-],NaN,positive stringent
8821,NaN,NaN,999-55-3,ALLYL ACRYLATE,NaN,NaN,NaN,NaN,NaN,0.0,21.0,NaN,0.0,0.0,1.0,1.0,0,0,1,1,",",C=CCOC(=O)C=C,NaN,negative stringent
8822,NaN,NaN,999-97-3,HEXAMETHYLDISILAZANE,NaN,NaN,NaN,NaN,NaN,0.0,19.0,NaN,0.0,0.0,1.0,1.0,0,0,1,1,",",C[Si](C)(C)N[Si](C)(C)C,NaN,negative stringent


In [4]:
df[['ratio_of_positive_carc', 'number_of_experiments_carc', 'ratio_of_positive_mut', 'number_of_experiments_mut']]
df['num_pos_experiments_carc'] = df['ratio_of_positive_carc'].values * df['number_of_experiments_carc'].values
df['num_pos_experiments_mut'] = df['ratio_of_positive_mut'].values * df['number_of_experiments_mut'].values

df['total_num_exp'] = df['number_of_experiments_mut'].fillna(0).values + df['number_of_experiments_carc'].fillna(0).values
df['total_ratio'] = (
    (df['num_pos_experiments_mut'].fillna(0).values + df['num_pos_experiments_carc'].fillna(0).values) /
    df['total_num_exp']
)

weight=5
df['total_num_exp_weighted'] = df['number_of_experiments_mut'].fillna(0).values + df['number_of_experiments_carc'].fillna(0).values * weight
df['total_ratio_weighted'] = (
    (df['num_pos_experiments_mut'].fillna(0).values + df['num_pos_experiments_carc'].fillna(0).values * weight) /
    df['total_num_exp_weighted']
)


In [5]:
print(df['mutagenic_label'].value_counts())
print(df['carcinogenic_label'].value_counts())

negative stringent    2936
positive stringent    1905
positive               862
negative               765
Name: mutagenic_label, dtype: int64
positive stringent    543
negative              304
negative stringent    295
positive              236
Name: carcinogenic_label, dtype: int64


In [ ]:
cpdb.df['smiles'].isin(df[df['ratio_of_positive_mut'].notnull()]['smiles']).sum()

In [ ]:
cpdb.df['smiles'].isin(df[df['ratio_of_positive_carc'].notnull()]['smiles']).sum()

In [ ]:
def plot_percent_overlap_cpdb(
    metric,
    experiments_column_name,
    plot_title='CCRIS Mutagenic Experiments overlap with CPDB',
    df=df,
    threshold_direction='less_than',
    min_number_experiments=1,
    show_plot=False
):
    assert threshold_direction in ['greater_than', 'less_than']

    set_of_breaks = set(np.nanpercentile(df[metric].values, np.arange(0, 100, 1)))
    if threshold_direction=='less_than':
        thresholds = sorted(list(set_of_breaks))
    else:
        thresholds = sorted(list(set_of_breaks), reverse=True)
        
    data = []

    full_vc = cpdb.df[cpdb.df['smiles'].isin(df['smiles'])]['class'].value_counts()

    for threshold in thresholds:
        if threshold_direction == 'less_than':
            temp = df[
                (df[metric] <= threshold) &
                (df[experiments_column_name] >= min_number_experiments)]
        else:
            temp = df[
                (df[metric] >= threshold) &
                (df[experiments_column_name] >= min_number_experiments)]
            
        vc = cpdb.df[cpdb.df['smiles'].isin(temp['smiles'])]['class'].value_counts()
        data.append({
            'threshold': threshold,
            'num_positives_cpdb': vc[1],
            'num_negatives_cpdb': vc[0],
            'percent_positives_cpdb': vc[1]/full_vc[1],
            'percent_negatives_cpdb': vc[0]/full_vc[0],
            'num_resulting_ccris_samples': len(temp),
            'num_required_experiments': min_number_experiments,
        })

    m_df = pd.DataFrame(data)
    m_df

    plot = plt.figure(figsize=(10, 5));
    plt.ylim(0,1);

    sns.histplot(
        x=metric,
        data=df[df[metric].notnull()],
        linewidth=3,
        alpha= 0.7,
        color= "g",
        binwidth=.05,
        stat='probability',
    #     cumulative=True
    );
    plt.plot(m_df['threshold'], m_df['percent_positives_cpdb'], label='percent positives CPDB');
    plt.plot(m_df['threshold'], m_df['percent_negatives_cpdb'], label='percent negatives CPDB');
    plt.xlabel('Threshold on % of positives to all from CCRIS');
    plt.ylabel('% of samples included from CPDB');
    plt.title(f'n experiments={min_number_experiments}');
    plt.suptitle(plot_title)
    plt.legend();
    plt.tight_layout()
    if not show_plot:
        plt.close()
    return m_df, plot

metric = 'ratio_of_positive_mut'
m_df, plot = plot_percent_overlap_cpdb(
    metric=metric,
    experiments_column_name='number_of_experiments_mut',
    min_number_experiments=1,
    show_plot=True
)

## Setting a limit on minimum number of experiments does not seem to decrease percentage of negatives to positives

In [ ]:

metric = 'ratio_of_positive_mut'
experiments_column_name='number_of_experiments_mut'
m_dfs = []
for min_number_experiments in np.arange(1, 10, 2):
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        plot_title='CCRIS Mutagenic Negatives Overlap with CPDB'
    )
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_positives_cpdb'] / m_dfs['num_negatives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 1000].sort_values('percent_overlap').iloc[:10]

In [ ]:
m_df, plot = plot_percent_overlap_cpdb(
    metric='ratio_of_positive_mut',
    experiments_column_name='number_of_experiments_mut',
    min_number_experiments=1,
    plot_title='CCRIS Mutagenic Negatives Overlap with CPDB',
    show_plot=True
)
plt.plot([0, 0], [0,1], c='red')

In [ ]:
metric = 'ratio_of_positive_carc'
experiments_column_name='number_of_experiments_carc'
m_dfs = []

for min_number_experiments in np.arange(1, 5, 1):
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        plot_title='CCRIS Carcinogenic Negatives Overlap with CPDB'
    )
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_positives_cpdb'] / m_dfs['num_negatives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 500].sort_values('percent_overlap').iloc[:10]

In [ ]:
m_df, plot = plot_percent_overlap_cpdb(
    metric='ratio_of_positive_carc',
    experiments_column_name='number_of_experiments_carc',
    min_number_experiments=1,
    plot_title='CCRIS Carcinogenic Negatives Overlap with CPDB',
    show_plot=True
)
plt.plot([0, 0], [0,1], c='red')

In [ ]:
metric = 'total_ratio'
experiments_column_name='total_num_exp'
m_dfs =[]

for min_number_experiments in np.arange(1, 20, 2)[::-1]:
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        plot_title='CCRIS Negatives Ratio Overlap with CPDB'
    )
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_positives_cpdb'] / m_dfs['num_negatives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:10]

In [ ]:
w_mdfs = []
for weight in [0.1, 0.5, 1, 2, 3, 5, 10, 15, 20, 30, 50, 100, 500, 1000, 10000]:
    df['total_num_exp_weighted'] = df['number_of_experiments_mut'].fillna(0).values + df['number_of_experiments_carc'].fillna(0).values * weight
    df['total_ratio_weighted'] = (
        (df['num_pos_experiments_mut'].fillna(0).values + df['num_pos_experiments_carc'].fillna(0).values * weight) /
        df['total_num_exp_weighted']
    )

    metric = 'total_ratio_weighted'
    experiments_column_name='total_num_exp_weighted'
    m_dfs =[]

    for min_number_experiments in np.arange(1, 11, 1):
        m_df, plot = plot_percent_overlap_cpdb(
            metric=metric,
            experiments_column_name=experiments_column_name,
            min_number_experiments=min_number_experiments,
            plot_title='CCRIS Negatives Weighted Ratio Overlap with CPDB'

        )
        m_dfs.append(m_df)

    m_dfs = pd.concat(m_dfs)
    m_dfs['percent_overlap'] = m_dfs['num_positives_cpdb'] / m_dfs['num_negatives_cpdb']
    m_dfs[m_dfs['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:10]
    m_dfs['weight'] = weight
    w_mdfs.append(m_dfs[m_dfs['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:20])
    
w_mdfs = pd.concat(w_mdfs)
w_mdfs[w_mdfs['num_resulting_ccris_samples'] > 3000].sort_values('percent_overlap').iloc[:20]

## Positives selection

In [ ]:
metric = 'ratio_of_positive_mut'
experiments_column_name='number_of_experiments_mut'
m_dfs = []
for min_number_experiments in np.arange(1, 10, 1)[::-1]:
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        threshold_direction='greater_than',
        plot_title='CCRIS Mutagenic Positives Overlap with CPDB'
    )
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_negatives_cpdb'] / m_dfs['num_positives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 500].sort_values('percent_overlap').iloc[:10]

In [ ]:
m_df, plot = plot_percent_overlap_cpdb(
    metric='ratio_of_positive_mut',
    experiments_column_name='number_of_experiments_mut',
    min_number_experiments=6,
    threshold_direction='greater_than',
    plot_title='CCRIS Mutagenic Positives Overlap with CPDB',
    show_plot=True
)
plt.plot([0.55, 0.55], [0,1], c='red')

In [ ]:
metric = 'ratio_of_positive_carc'
experiments_column_name='number_of_experiments_carc'
m_dfs = []
for min_number_experiments in np.arange(1, 5, 1):
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        threshold_direction='greater_than',
        plot_title='CCRIS Carcinogenic Positives Overlap with CPDB'
    )
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_negatives_cpdb'] / m_dfs['num_positives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 500].sort_values('percent_overlap').iloc[:10]

In [ ]:
m_df, plot = plot_percent_overlap_cpdb(
    metric='ratio_of_positive_carc',
    experiments_column_name='number_of_experiments_carc',
    min_number_experiments=2,
    threshold_direction='greater_than',
    plot_title='CCRIS Carcinogenic Positives Overlap with CPDB',
    show_plot=True
)
plt.plot([0.6, 0.6], [0,1], c='red')

In [ ]:
metric = 'total_ratio'
experiments_column_name='total_num_exp'
m_dfs = []
for min_number_experiments in np.arange(1, 10, 1):
    m_df, plot = plot_percent_overlap_cpdb(
        metric=metric,
        experiments_column_name=experiments_column_name,
        min_number_experiments=min_number_experiments,
        threshold_direction='greater_than',
        plot_title='CCRIS Positives Ratio Overlap with CPDB'
    )
    m_dfs.append(m_df)

m_dfs = pd.concat(m_dfs)
m_dfs['percent_overlap'] = m_dfs['num_negatives_cpdb'] / m_dfs['num_positives_cpdb']
m_dfs[m_dfs['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:20]

In [ ]:
w_mdfs_pos = []
for weight in [0.1, 0.5, 1, 2, 3, 5, 10, 15, 20, 30, 50, 100, 500, 1000]:
    df['total_num_exp_weighted'] = df['number_of_experiments_mut'].fillna(0).values + df['number_of_experiments_carc'].fillna(0).values * weight
    df['total_ratio_weighted'] = (
        (df['num_pos_experiments_mut'].fillna(0).values + df['num_pos_experiments_carc'].fillna(0).values * weight) /
        df['total_num_exp_weighted']
    )

    metric = 'total_ratio_weighted'
    experiments_column_name='total_num_exp_weighted'
    m_dfs = []
    for min_number_experiments in np.arange(1, 10, 1):
        m_df, plot = plot_percent_overlap_cpdb(
            metric=metric,
            experiments_column_name=experiments_column_name,
            min_number_experiments=min_number_experiments,
            threshold_direction='greater_than',
            plot_title='CCRIS Positives Weighted Ratio Overlap with CPDB'
        )
        m_dfs.append(m_df)

    m_dfs = pd.concat(m_dfs)
    m_dfs['percent_overlap'] = m_dfs['num_negatives_cpdb'] / m_dfs['num_positives_cpdb']
    m_dfs['weight'] = weight
    w_mdfs_pos.append(m_dfs[m_dfs['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:20])
    
w_mdfs_pos = pd.concat(w_mdfs_pos)
w_mdfs_pos[w_mdfs_pos['num_resulting_ccris_samples'] > 3000].sort_values('percent_overlap').iloc[:20]

In [ ]:
w_mdfs_pos[w_mdfs_pos['num_resulting_ccris_samples'] > 2000].sort_values('percent_overlap').iloc[:20]

## Compare mutagenicity datasets

### Compose canonical datasets:

In [ ]:
cpdb.df

In [ ]:
from dgl.data import TUDataset
data = TUDataset('Mutagenicity')

In [6]:
ames_df = pd.read_csv('../data/hansen_2009_ames.smi', sep='\t', names=['smiles', 'cas', 'class'])
ames_df['smiles'] = GraphCancerMolecules.smiles_standardize(ames_df['smiles'])
ames_df = ames_df[ames_df['smiles'].notnull()]

print(f"number of smiles null {ames_df['smiles'].isnull().sum()}")
print(f"Number of ames data in CCRIS {ames_df['smiles'].isin(df['smiles']).sum()}")
print(f"Number of CCRIS data in ames dataset {df['smiles'].isin(ames_df['smiles']).sum()}")
print(len(ames_df), len(df))

number of smiles null 0
Number of ames data in CCRIS 5084
Number of CCRIS data in ames dataset 5318
6506 8126


In [7]:
drop = True

carc_neg = df[(df['ratio_of_positive_carc'] <= 0) & (df['number_of_experiments_carc'] >= 1)][['smiles']]
carc_neg['ccris_class'] = 0
carc_pos = df[(df['ratio_of_positive_carc'] >= 0.6) & (df['number_of_experiments_carc'] >= 2)][['smiles']]
carc_pos['ccris_class'] = 1
carc_ccris = pd.concat([carc_neg, carc_pos])
carc_ccris['source'] = 'ccris'
carc = pd.merge(carc_ccris, cpdb.df, on='smiles', how='outer')

ct = pd.crosstab(carc['ccris_class'], carc['class'])
n_samples = ct.sum().sum()
print(ct.to_string())
print((ct/n_samples).to_string())

if drop:
    carc = carc.drop(index=carc[
        (carc['ccris_class'].notnull())
        & (carc['class'].notnull())
        & (carc['class'] != carc['ccris_class'])
    ].index)
else:
    # use if positive is positive class strategy
    carc.loc[(carc['ccris_class'].notnull())
        & (carc['class'].notnull())
        & (carc['class'] != carc['ccris_class']), 'class'] = 1
    
carc.loc[carc['class'].isnull(), 'class'] = carc.loc[carc['class'].isnull()]['ccris_class']
carc['source'] = carc['source_x'].fillna("") + "," + carc['source_y'].fillna("")
carc['source'] = carc['source'].str.strip(',')
carc = carc.drop(columns=['source_y', 'source_x', 'ccris_class'])
carc


NameError: name 'cpdb' is not defined

In [ ]:
mut_pos = df[(df['ratio_of_positive_mut'] <= 0) & (df['number_of_experiments_mut'] >= 1)][['smiles']]
mut_pos['class'] = 0
mut_neg = df[(df['ratio_of_positive_mut'] >= 0.54) & (df['number_of_experiments_mut'] >= 6)][['smiles']]
mut_neg['class'] = 1
mut_ccris = pd.concat([mut_pocs, mut_neg])
mut_ccris['source'] = 'ccris'
mut_ccris

In [ ]:
print(ames_df['class'].value_counts())
print(mut_ccris['class'].value_counts())
print(carc['class'].value_counts())

In [ ]:
mut = ames_df.merge(mut_ccris, on='smiles', how='outer', suffixes=['_ames', '_ccris'])
ct = pd.crosstab(mut['class_ames'], mut['class_ccris'])
n_samples = ct.sum().sum()
print(ct.to_string())
print((ct/n_samples).to_string())

In [ ]:
mut_carc = ames_df.merge(carc, on='smiles', how='outer', suffixes=['_ames', '_carc'])
ct = pd.crosstab(mut_carc['class_ames'], mut_carc['class_carc'])
n_samples = ct.sum().sum()
print(ct.to_string()+'\n')
print((ct/n_samples).to_string())

In [ ]:
mut_carc = mut_ccris.merge(carc, on='smiles', how='outer', suffixes=['_mut', '_carc'])
ct = pd.crosstab(mut_carc['class_mut'], mut_carc['class_carc'])
n_samples = ct.sum().sum()
print(ct.to_string()+'\n')
print((ct/n_samples).to_string())

## Compare with Drug Data Bank

In [ ]:
ddb = pd.read_csv('../data/ddb_approved.csv')
drug_df = pd.read_csv('../data/potentially_carcinogenic_drugs.csv')
print(len(ddb))
# Remove drugs that are either illicit or withdrawn
ddb = ddb[(~ddb['Drug Groups'].str.contains('withdrawn')) & (~ddb['Drug Groups'].str.contains('illicit'))]
# remove chemotherapeutics and nucleoside analogues 
ddb = ddb[~ddb['Name'].str.lower().isin(drug_df['drug_name'].str.lower())]
ddb['smiles'] = GraphCancerMolecules.smiles_standardize(ddb['SMILES'])
ddb = ddb[ddb['smiles'].notnull()]
print(len(ddb))
ddb = ddb[['Name', 'smiles','DrugBank ID', 'Drug Groups', 'CAS Number']]
ddb['class'] = 0

print("\n\nAmes intersection with DDB:")
mut_ddb = ames_df.merge(ddb, on='smiles', how='outer', suffixes=['_ames', '_ddb'])
ct = pd.crosstab(mut_ddb['class_ames'], mut_ddb['class_ddb'])
n_samples = ct.sum().sum()
print(ct.to_string()+'\n')
print((ct/n_samples).to_string())

print("\n\nCarciongenicity intersection with DDB:")
carc_ddb = carc.merge(ddb, on='smiles', how='outer', suffixes=['_carc', '_ddb'])
ct = pd.crosstab(carc_ddb['class_carc'], carc_ddb['class_ddb'])
n_samples = ct.sum().sum()
print(ct.to_string()+'\n')
print((ct/n_samples).to_string())

In [ ]:
temp = carc_ddb[(carc_ddb['class_carc'].notnull()) & (carc_ddb['class_ddb'].notnull())]
temp = temp[temp['class_carc'] != temp['class_ddb']]

### CPDB comparison

In [ ]:
cpdb_lit = pd.read_csv('../data/cpdb.lit.tab.txt', sep='\t')
cpdb_nci = pd.read_csv('../data/cpdb.ncintp.tab.txt', sep='\t')
cpdb_df = pd.concat([cpdb_lit, cpdb_nci])

print(cpdb_df[cpdb_df['species']=='r']['chemcode'].nunique())
print(cpdb_df['chemcode'].nunique())
cpdb_name = pd.read_csv('../data/cpdb_name.tsv', sep='\t')
assert not cpdb_name['chemcode'].duplicated().sum()
cpdb_df_len = len(cpdb_df)
cpdb_df = cpdb_df.merge(cpdb_name, on='chemcode')
assert cpdb_df_len == len(cpdb_df)
cpdb_df['td50_log'] = np.log10(cpdb_df['td50'])
cpdb_df

#### Visualize number of experiments per molecule

In [ ]:
plt.figure(figsize=(20,5))
vc = cpdb_df['chemcode'].value_counts()
print(pd.Series(vc.values).describe())
print(f"number of compounds with more than 50 studies: {len(vc[vc>50])}")
sns.distplot(vc[vc<=50],kde=False, bins=50)
plt.xticks(np.arange(1,51, 1))
plt.title('Number of experiments per compound')
plt.xlim(0,50)


In [ ]:

plt.figure(figsize=(10,5))
plt.title('Distribution of TD50 values per animal')
print(cpdb_df['species'].value_counts())
ax = sns.violinplot(data=cpdb_df[cpdb_df['species'].isin(['m', 'r', 'h'])],x='species', y='td50_log')
sns.stripplot(data=cpdb_df[cpdb_df['species'].isin(['m', 'r', 'h'])],x='species', y='td50_log',linewidth=1, color='white',alpha=.5)

ax.set_xticklabels(['rat', 'mouse', 'hamster'])

#### Have to decide on an aggregation technique

In [ ]:
max_td50 = cpdb_df[['chemcode', 'td50_log', 'ctotal', 'ctumors', 'datanum',]].groupby('chemcode').max()['td50_log']
min_td50 = cpdb_df[['chemcode', 'td50_log', 'ctotal', 'ctumors', 'datanum',]].groupby('chemcode').min()['td50_log']

(max_td50-min_td50).describe()

In [ ]:
from scipy.stats import hmean
cpdb_g = cpdb_df[~cpdb_df['cas'].duplicated()][['name', 'cas','chemcode', 'species']]
cpdb_g_len = len(cpdb_g)

temp = cpdb_df[['cas', 'td50_log', 'td50',]].groupby('cas').min().reset_index()
columns_t = temp.columns.tolist()
columns_t.remove('cas')
columns_t = {x:x+'_min' for x in columns_t}
temp.rename(columns=columns_t, inplace=True)
cpdb_g = pd.merge(cpdb_g, temp, on='cas')

temp = cpdb_df[['cas', 'td50_log', 'td50',]].groupby('cas').mean().reset_index()
columns_t = temp.columns.tolist()
columns_t.remove('cas')
columns_t = {x:x+'_mean' for x in columns_t}
temp.rename(columns=columns_t, inplace=True)
cpdb_g = pd.merge(cpdb_g, temp, on='cas')

temp = cpdb_df[['cas', 'td50_log', 'td50',]].groupby('cas').median().reset_index()
columns_t = temp.columns.tolist()
columns_t.remove('cas')
columns_t = {x:x+'_median' for x in columns_t}
temp.rename(columns=columns_t, inplace=True)
cpdb_g = pd.merge(cpdb_g, temp, on='cas')
assert cpdb_g_len == len(cpdb_g)

temp = cpdb_df[['cas', 'td50',]].copy()
temp = temp.groupby('cas').apply(hmean)
temp = temp.reset_index()
temp.rename(columns={0: 'td50_harmonic'}, inplace=True)
temp['td50_harmonic'] = [x[0] for x in temp['td50_harmonic']]
cpdb_g = cpdb_g.merge(temp, on='cas')

temp = cpdb_df[['cas', 'td50',]].copy()
temp['td50'] = np.log(temp['td50']+ 1)
temp = temp.groupby('cas').apply(hmean)
temp = temp.reset_index()
temp.rename(columns={0: 'td50_log_harmonic'}, inplace=True)
temp['td50_log_harmonic'] = [x[0] for x in temp['td50_log_harmonic']]
cpdb_g = cpdb_g.merge(temp, on='cas')

assert cpdb_g_len ==len(cpdb_g)

exclude_cas = ['---', 'mixture']
cpdb_g = cpdb_g[~cpdb_g['cas'].isin(exclude_cas)]


In [ ]:
import cirpy
import datetime

smiles_data = []

for index, row in cpdb_g.iterrows():
    if index%100 == 0:
        print(str(datetime.datetime.now()), index)
    
    try:
        smiles = cirpy.resolve(row['cas'], 'smiles', ['cas_number', 'name_by_cir', 'name_by_opsin'])
        smiles_data.append({'smiles': smiles, 'cas':row['cas']})
    except Exception as error:
        print(error)
        continue

len_old_df = len(cpdb_g)
cpdbg = pd.merge(
    cpdb_g,
    pd.DataFrame(smiles_data),
    on='cas',
    how='outer',
)
assert len(cpdbg) == len_old_df
cpdbg['smiles'] = GraphCancerMolecules.smiles_standardize(cpdbg['smiles'])
cpdbg = cpdbg[~cpdbg['smiles'].isnull()]

#### Visualize DDB overlap as a function of TD50

In [ ]:
threshold_direction = 'less_than'
metric_names=['td50_log_min','td50_min', 'td50_log_mean', 'td50_mean', 'td50_log_median', 'td50_median', 'td50_harmonic', 'td50_log_harmonic']
m_dfs = []
for metric in metric_names:
    set_of_breaks = set(np.nanpercentile(cpdb_g[metric].values, np.arange(0, 100, 1)))
    thresholds = sorted(list(set_of_breaks))
    data=[]

    for threshold in thresholds:
        if threshold_direction == 'less_than':
            temp = cpdbg[
                (cpdbg[metric] <= threshold) 
    #             & (cpdb_g[experiments_column_name] >= min_number_experiments)
            ]
        else:
            temp = cpdb_g[
                (cpdb_g[metric] >= threshold)
    #             & (cpdb_g[experiments_column_name] >= min_number_experiments)
            ]

        n_overlap = len(ddb[ddb['smiles'].isin(temp['smiles'])])
        data.append({
            metric: threshold,
            'ddb_overlap': n_overlap,
            'len_cpdb': len(temp),
    #         'num_required_experiments': min_number_experiments,
        })

    m_df = pd.DataFrame(data)
    m_df['fraction_overlap'] = m_df['ddb_overlap'] / m_df['ddb_overlap'].max()
    m_df['fraction_cpdb'] = m_df['len_cpdb'] / len(cpdbg)
    m_dfs.append(m_df)
    
m_dfs = pd.concat(m_dfs)

In [ ]:
plot = plt.figure(figsize=(20, 10));
metric_names=['td50_log_min','td50_min', 'td50_log_mean', 'td50_mean', 'td50_log_median', 'td50_median', 'td50_harmonic', 'td50_log_harmonic']

for metric_val in [metric_names[2], metric_names[1], metric_names[7]]:
    temp = m_dfs[m_dfs[metric_val].notnull()]
    if 'log' not in metric:
        temp[metric] = np.log(temp[metric_val].values)
    plt.plot(temp['fraction_overlap'], temp['fraction_overlap']/temp['fraction_cpdb'], label=f'ratio of DDB overlap to CPDB percentage {metric_val}')
# sns.histplot(
#     x=metric,
#     data=m_df[m_df[metric].notnull()],
#     linewidth=1,
#     alpha= 0.7,
#     color= "g",
#     binwidth=1,
#     stat='probability',
# #     cumulative=True,
#     label='distribution of CPDB log TD50 values'
# );
plt.ylabel("")
plt.title(f"Molecule overlap n={m_df['ddb_overlap'].max()} between DDB and CPDB\n as a function of td50")
plt.legend()

## Validate harmonic mean aggregations on their website

In [ ]:
cpdbs = pd.read_csv('/Users/phil/Downloads/CPDBChemical.csv',
                    names=['name', 'cas', 'ames', 'rat_td50', 'mouse_td50', 'rat_target_m', 'rat_target_f', 'mouse_target_m', 'mouse_target_f'], 
                    skiprows=2)

def define_rat_class(td50_value):
    if td50_value in ['.', 'I']:
        return None
    elif td50_value in ['–', '-']:
        return 0
    else: 
        return 1
cpdbs = cpdbs[~cpdbs['rat_td50'].isnull()]
cpdbs['rat_class'] = [define_rat_class(x) for x in cpdbs['rat_td50'].values]
cpdbs['mouse_class'] = [define_rat_class(x) for x in cpdbs['mouse_td50'].values]

fields_to_clean = ['mouse_td50', 'rat_td50']
for field_to_clean in fields_to_clean:
    cpdbs[field_to_clean] = [''.join([value for value in td50 if not value.isalpha()]) for td50 in cpdbs[field_to_clean].values]
    cpdbs[field_to_clean] = cpdbs[field_to_clean].str.replace(',', '')
    cpdbs[field_to_clean] = cpdbs[field_to_clean].str.replace('<', '')
    cpdbs[field_to_clean].replace('.', np.nan, inplace=True)
    cpdbs[field_to_clean].replace('', np.nan, inplace=True)
    cpdbs[field_to_clean].replace('–', 100000, inplace=True)

    cpdbs[field_to_clean] = [float(x) for x in cpdbs[field_to_clean].values]

cpdbg2 = cpdbg.merge(cpdbs[['rat_class', 'mouse_class', 'rat_td50', 'mouse_td50', 'cas']], on='cas', how='left')

In [ ]:
ct = pd.crosstab(cpdbg2['rat_class'].values, cpdbg2['mouse_class'].values)
n_samples = ct.sum().sum()
print(ct.to_string())
print((ct/n_samples).to_string())

In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind, pearsonr, spearmanr

In [ ]:
plt.figure(figsize=(7,5))
xval = 'mouse_td50'
yval = 'td50_log_harmonic'
temp = cpdbg2[(cpdbg2[xval].notnull()) & (cpdbg2[yval].notnull())]
per = pearsonr(temp[xval], temp[yval])
print(per)
spr = spearmanr(temp[xval], temp[yval])
print(spr)
sns.kdeplot(temp[xval].values, temp[yval].values, cut=0, shade_lowest=True, 
            fill=True, alpha=0.7, cmap="mako", label=f'spearman = {spr[0]:.2f}', log_scale=True)
plt.xlabel('Mouse td50')
plt.legend(loc='upper left')
plt.ylabel('TD50 log harmonic')
plt.suptitle(f'Correlation between mouse and Harmonic Aggreated TD50')
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
xval = 'rat_td50'
yval = 'td50_log_harmonic'
temp = cpdbg2[(cpdbg2[xval].notnull()) & (cpdbg2[yval].notnull())]
per = pearsonr(temp[xval], temp[yval])
print(per)
spr = spearmanr(temp[xval], temp[yval])
print(spr)
sns.kdeplot(temp[xval].values, temp[yval].values, cut=0, shade_lowest=True, 
            fill=True, alpha=0.7, cmap="mako", label=f'spearman = {spr[0]:.2f}', log_scale=True)
plt.xlabel('rat td50')
plt.legend(loc='upper left')
plt.ylabel('TD50 log harmonic')
plt.suptitle(f'Correlation between rat and Harmonic Aggreated TD50')
plt.show()

In [ ]:
plt.figure(figsize=(7,5))
xval = 'mouse_td50'
yval = 'rat_td50'
temp = cpdbg2[(cpdbg2[xval].notnull()) & (cpdbg2[yval].notnull())]
per = pearsonr(temp[xval], temp[yval])
print(per)
spr = spearmanr(temp[xval], temp[yval])
print(spr)
sns.kdeplot(temp[xval].values, temp[yval].values, cut=0, shade_lowest=True, 
            fill=True, alpha=0.7, cmap="mako", label=f'spearman = {spr[0]:.2f}', log_scale=True)
plt.xlabel('Mouse td50')
plt.legend(loc='upper left')
plt.ylabel('Rat td50')
plt.suptitle(f'Correlation between mouse and rat TD50 CPDB')
plt.show()

In [ ]:
temp = cpdbg.merge(cpdb.df, on='smiles')
sns.violinplot(x="class", y="td50_log_mean", data=temp)
plt.show()


In [ ]:
temp = cpdbg.merge(cpdb.df, on='smiles')
sns.violinplot(x="class", y="td50_log_mean", data=temp)
plt.show()
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(temp['class'], temp['td50_log_mean'], pos_label=0)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('TD50 log discriminatory performance for CarcinoPredEL')
plt.plot(fpr, tpr, label=f'AUC {round(metrics.auc(fpr, tpr),3)}')

fpr, tpr, thresholds = metrics.roc_curve(temp['class'], temp['td50_log_median'], pos_label=0)
plt.plot(fpr, tpr, label=f'AUC {round(metrics.auc(fpr, tpr),3)}')
plt.legend()
fpr, tpr, thresholds = metrics.roc_curve(temp['class'], temp['td50_log_min'], pos_label=0)
plt.plot(fpr, tpr, label=f'AUC {round(metrics.auc(fpr, tpr),3)}')
plt.legend()